# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re


In [42]:
def retrieve_titles(book_soup):
    titles = book_soup.findAll('h3')
    final_titles = [book.find('a').attrs['title'] for book in book_soup.findAll('h3')]
#     print(final_titles)
    return final_titles

SyntaxError: invalid syntax (Temp/ipykernel_23360/11285091.py, line 5)

In [37]:
retrieve_titles(book_soup)

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

In [43]:
print(final_titles)

NameError: name 'final_titles' is not defined

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [53]:
def retrieve_ratings(book_soup):
    star_ratings = []
    star_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    regex = re.compile("star-rating (.*)")
    for r in book_soup.findAll('p', {"class" : regex}):
        star_ratings.append(r.attrs['class'][-1])
    star_ratings = [star_dict[s] for s in star_ratings]
#     print(star_ratings)
    return star_ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [39]:
def retrieve_prices(book_soup):
    prices = [float(p.text[1:]) for p in book_soup.findAll('p', class_="price_color")]
#     print(prices)
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [40]:
def retrieve_availabilities(book_soup):
    avails = [a.text.strip() for a in book_soup.findAll('p', class_="instock availability")]
#     print(avails)
    return avails

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [6]:
html_page = requests.get('http://books.toscrape.com/')

soup = BeautifulSoup(html_page.content, 'html.parser')


In [65]:
#Your code here
final_titles = []
star_ratings = []
prices = []
avails = []

for i in range(1,51):
    if i == 1:
        url = 'http://books.toscrape.com/'
    else:
        url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
#     book_soup = soup.find('ol', class_="row")
    final_titles += retrieve_titles(book_soup)       #final_titles, list of strings
    star_ratings += retrieve_ratings(book_soup)      #ratings, lsit of ints
    prices += retrieve_prices(book_soup)             #prices, list of floats
    avails += retrieve_availabilities(book_soup)     #avails, list of strings


book_frame = pd.DataFrame([final_titles, star_ratings, prices, avails]).transpose()
book_frame.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
book_frame


,Title,Star_Rating,Price_(pounds),Availability
0,Frankenstein,2,38.0,In stock
1,Forever Rockers (The Rocker #12),3,28.8,In stock
2,Fighting Fate (Fighting #6),3,39.24,In stock
3,Emma,2,32.93,In stock
4,"Eat, Pray, Love",3,51.32,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock


20

## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [72]:
#Your code here
# final_titles = []
# star_ratings = []
# prices = []
# avails = []

def get_next_page(soup):
    next_button = soup.find("li", class_="next") #May return none if on final page
    if next_button:
        return next_button.find('a').attrs['href']
    else:
        return None

def parse_url(url, final_titles=[], star_ratings=[], prices=[], avails=[]):
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
#     book_soup = soup.find('ol', class_="row")
    final_titles += retrieve_titles(book_soup)       #final_titles, list of strings
    star_ratings += retrieve_ratings(book_soup)      #ratings, lsit of ints
    prices += retrieve_prices(book_soup)             #prices, list of floats
    avails += retrieve_availabilities(book_soup)     #avails, list of strings
    
    next_url = get_next_page(soup)
    if next_url:
        next_url = '/'.join(url.split('/')[:-1])+'/' + next_url
        return parse_url(next_url, final_titles, star_ratings, prices, avails)
    else:
        return final_titles, star_ratings, prices, avails

url = 'http://books.toscrape.com/'
final_titles, star_ratings, prices, avails = parse_url(url, final_titles, star_ratings, prices, avails)
        
book_frame = pd.DataFrame([final_titles, star_ratings, prices, avails]).transpose()
book_frame.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
book_frame

,Title,Star_Rating,Price_(pounds),Availability
0,Frankenstein,2,38.0,In stock
1,Forever Rockers (The Rocker #12),3,28.8,In stock
2,Fighting Fate (Fighting #6),3,39.24,In stock
3,Emma,2,32.93,In stock
4,"Eat, Pray, Love",3,51.32,In stock
...,...,...,...,...
3015,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
3016,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
3017,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
3018,1st to Die (Women's Murder Club #1),1,53.98,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!